# Data Observability

Data observability is the ability to understand the health and state of data in your system. It extends traditional software observability concepts to the data layer, enabling teams to identify, troubleshoot, and resolve data issues before they impact downstream consumers.

## Why Data Observability Matters

- **Trust**: Stakeholders need confidence that data is accurate and reliable
- **Speed**: Quickly detect and resolve data issues before they propagate
- **Scale**: Monitor thousands of tables and pipelines automatically
- **Collaboration**: Bridge gaps between data engineers, analysts, and consumers

## The Five Pillars of Data Observability

Data observability is built on five core pillars that together provide comprehensive visibility into data health:

| Pillar | Description | Key Questions |
|--------|-------------|---------------|
| **Freshness** | Is the data up-to-date? | When was the table last updated? Is it within SLA? |
| **Volume** | Is the data complete? | Are row counts within expected ranges? Any missing partitions? |
| **Schema** | Has the structure changed? | Are there new/removed columns? Data type changes? |
| **Distribution** | Are values within expected ranges? | Are there null spikes? Statistical anomalies? |
| **Lineage** | Where does data come from and go? | What upstream/downstream dependencies exist? |

## 1. Freshness Monitoring

Freshness tracks when data was last updated. Stale data can lead to incorrect business decisions.

In [ ]:
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Optional
import pandas as pd

@dataclass
class FreshnessCheck:
    """Monitor data freshness against SLA thresholds."""
    table_name: str
    expected_update_frequency: timedelta
    last_updated: datetime
    
    def is_fresh(self) -> bool:
        """Check if data is within freshness SLA."""
        age = datetime.now() - self.last_updated
        return age <= self.expected_update_frequency
    
    def staleness_hours(self) -> float:
        """Return how many hours past SLA the data is."""
        age = datetime.now() - self.last_updated
        overdue = age - self.expected_update_frequency
        return max(0, overdue.total_seconds() / 3600)
    
    def status_report(self) -> dict:
        """Generate freshness status report."""
        return {
            "table": self.table_name,
            "last_updated": self.last_updated.isoformat(),
            "expected_frequency_hours": self.expected_update_frequency.total_seconds() / 3600,
            "is_fresh": self.is_fresh(),
            "staleness_hours": self.staleness_hours()
        }

# Example usage
check = FreshnessCheck(
    table_name="orders",
    expected_update_frequency=timedelta(hours=1),
    last_updated=datetime.now() - timedelta(hours=2, minutes=30)
)

print(f"Is fresh: {check.is_fresh()}")
print(f"Staleness: {check.staleness_hours():.2f} hours past SLA")

## 2. Volume Monitoring

Volume monitoring ensures data completeness by tracking row counts and detecting anomalies like missing data or unexpected spikes.

In [ ]:
import numpy as np
from typing import List, Tuple

class VolumeMonitor:
    """Monitor data volume for anomalies using statistical methods."""
    
    def __init__(self, historical_counts: List[int], z_threshold: float = 3.0):
        """
        Args:
            historical_counts: List of historical row counts
            z_threshold: Number of standard deviations for anomaly detection
        """
        self.historical_counts = np.array(historical_counts)
        self.z_threshold = z_threshold
        self.mean = np.mean(self.historical_counts)
        self.std = np.std(self.historical_counts)
    
    def check_volume(self, current_count: int) -> Tuple[bool, str]:
        """Check if current volume is within expected range."""
        if self.std == 0:
            # Handle case with no variance
            is_anomaly = current_count != self.mean
            z_score = 0
        else:
            z_score = (current_count - self.mean) / self.std
            is_anomaly = abs(z_score) > self.z_threshold
        
        if is_anomaly:
            direction = "high" if z_score > 0 else "low"
            message = f"ANOMALY: Volume is unusually {direction} (z-score: {z_score:.2f})"
        else:
            message = f"Volume is within normal range (z-score: {z_score:.2f})"
        
        return not is_anomaly, message
    
    def get_expected_range(self) -> Tuple[float, float]:
        """Return expected volume range based on threshold."""
        lower = self.mean - (self.z_threshold * self.std)
        upper = self.mean + (self.z_threshold * self.std)
        return max(0, lower), upper

# Example: Historical daily row counts
historical = [10000, 10200, 9800, 10100, 10050, 9900, 10300, 10150]
monitor = VolumeMonitor(historical, z_threshold=2.5)

# Test different scenarios
test_counts = [10100, 5000, 15000, 0]
for count in test_counts:
    is_ok, message = monitor.check_volume(count)
    print(f"Count {count:,}: {message}")

## 3. Schema Monitoring

Schema changes can break downstream pipelines. Monitoring detects additions, removals, and type changes.

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, Set, List
from enum import Enum

class SchemaChangeType(Enum):
    COLUMN_ADDED = "column_added"
    COLUMN_REMOVED = "column_removed"
    TYPE_CHANGED = "type_changed"
    NULLABLE_CHANGED = "nullable_changed"

@dataclass
class ColumnInfo:
    name: str
    data_type: str
    nullable: bool = True

@dataclass
class SchemaChange:
    change_type: SchemaChangeType
    column_name: str
    old_value: str = None
    new_value: str = None
    
    def __str__(self):
        if self.change_type == SchemaChangeType.COLUMN_ADDED:
            return f"+ Column '{self.column_name}' added ({self.new_value})"
        elif self.change_type == SchemaChangeType.COLUMN_REMOVED:
            return f"- Column '{self.column_name}' removed"
        elif self.change_type == SchemaChangeType.TYPE_CHANGED:
            return f"~ Column '{self.column_name}' type: {self.old_value} -> {self.new_value}"
        elif self.change_type == SchemaChangeType.NULLABLE_CHANGED:
            return f"~ Column '{self.column_name}' nullable: {self.old_value} -> {self.new_value}"

class SchemaMonitor:
    """Detect and track schema changes between versions."""
    
    def __init__(self, baseline_schema: Dict[str, ColumnInfo]):
        self.baseline = baseline_schema
    
    def compare(self, current_schema: Dict[str, ColumnInfo]) -> List[SchemaChange]:
        """Compare current schema against baseline."""
        changes = []
        baseline_cols = set(self.baseline.keys())
        current_cols = set(current_schema.keys())
        
        # Detect removed columns
        for col in baseline_cols - current_cols:
            changes.append(SchemaChange(
                change_type=SchemaChangeType.COLUMN_REMOVED,
                column_name=col
            ))
        
        # Detect added columns
        for col in current_cols - baseline_cols:
            changes.append(SchemaChange(
                change_type=SchemaChangeType.COLUMN_ADDED,
                column_name=col,
                new_value=current_schema[col].data_type
            ))
        
        # Detect type/nullable changes
        for col in baseline_cols & current_cols:
            old = self.baseline[col]
            new = current_schema[col]
            
            if old.data_type != new.data_type:
                changes.append(SchemaChange(
                    change_type=SchemaChangeType.TYPE_CHANGED,
                    column_name=col,
                    old_value=old.data_type,
                    new_value=new.data_type
                ))
            
            if old.nullable != new.nullable:
                changes.append(SchemaChange(
                    change_type=SchemaChangeType.NULLABLE_CHANGED,
                    column_name=col,
                    old_value=str(old.nullable),
                    new_value=str(new.nullable)
                ))
        
        return changes

# Example usage
baseline = {
    "id": ColumnInfo("id", "INTEGER", nullable=False),
    "name": ColumnInfo("name", "VARCHAR(100)"),
    "email": ColumnInfo("email", "VARCHAR(255)"),
    "created_at": ColumnInfo("created_at", "TIMESTAMP")
}

current = {
    "id": ColumnInfo("id", "BIGINT", nullable=False),  # Type changed
    "name": ColumnInfo("name", "VARCHAR(100)"),
    "phone": ColumnInfo("phone", "VARCHAR(20)"),  # Added
    "created_at": ColumnInfo("created_at", "TIMESTAMP")
    # email removed
}

monitor = SchemaMonitor(baseline)
changes = monitor.compare(current)

print("Schema Changes Detected:")
print("-" * 40)
for change in changes:
    print(change)

## 4. Distribution Monitoring

Distribution monitoring tracks statistical properties of data values to detect drift, null rate changes, and outliers.

In [ ]:
from scipy import stats
import pandas as pd
import numpy as np

class DistributionMonitor:
    """Monitor data distribution for drift and anomalies."""
    
    def __init__(self, baseline_data: pd.Series):
        self.baseline = baseline_data.dropna()
        self.baseline_stats = self._compute_stats(self.baseline)
    
    def _compute_stats(self, data: pd.Series) -> dict:
        """Compute statistical summary."""
        return {
            "mean": data.mean(),
            "std": data.std(),
            "min": data.min(),
            "max": data.max(),
            "median": data.median(),
            "null_rate": 1 - (len(data) / len(self.baseline)),
            "unique_count": data.nunique()
        }
    
    def check_distribution(self, current_data: pd.Series, 
                          ks_threshold: float = 0.05) -> dict:
        """
        Check current data against baseline distribution.
        Uses Kolmogorov-Smirnov test for distribution comparison.
        """
        current = current_data.dropna()
        current_stats = self._compute_stats(current)
        
        # Perform KS test
        ks_statistic, p_value = stats.ks_2samp(self.baseline, current)
        
        # Check for significant drift
        has_drift = p_value < ks_threshold
        
        # Check null rate change
        original_null_rate = 1 - (len(current) / len(current_data))
        baseline_null_rate = 1 - (len(self.baseline) / len(self.baseline))
        null_rate_change = abs(original_null_rate - baseline_null_rate)
        
        return {
            "has_drift": has_drift,
            "ks_statistic": ks_statistic,
            "p_value": p_value,
            "baseline_stats": self.baseline_stats,
            "current_stats": current_stats,
            "null_rate_change": null_rate_change,
            "mean_shift": current_stats["mean"] - self.baseline_stats["mean"]
        }

# Example: Monitoring price distribution
np.random.seed(42)

# Baseline: Normal distribution around $100
baseline_prices = pd.Series(np.random.normal(100, 15, 1000))

# Current: Shifted distribution (prices increased)
current_prices = pd.Series(np.random.normal(115, 15, 1000))

monitor = DistributionMonitor(baseline_prices)
result = monitor.check_distribution(current_prices)

print("Distribution Analysis:")
print("-" * 40)
print(f"Drift Detected: {result['has_drift']}")
print(f"KS Statistic: {result['ks_statistic']:.4f}")
print(f"P-Value: {result['p_value']:.4e}")
print(f"Mean Shift: ${result['mean_shift']:.2f}")

## 5. Data Lineage

Lineage tracks data flow from source to destination, enabling impact analysis and root cause identification.

In [ ]:
from collections import defaultdict
from typing import Set, List, Tuple

class DataLineage:
    """Track data lineage relationships between datasets."""
    
    def __init__(self):
        self.upstream: Dict[str, Set[str]] = defaultdict(set)  # table -> sources
        self.downstream: Dict[str, Set[str]] = defaultdict(set)  # table -> consumers
        self.metadata: Dict[str, dict] = {}  # table -> metadata
    
    def add_table(self, table: str, metadata: dict = None):
        """Register a table with optional metadata."""
        if table not in self.upstream:
            self.upstream[table] = set()
        if metadata:
            self.metadata[table] = metadata
    
    def add_lineage(self, source: str, target: str):
        """Add lineage relationship: source -> target."""
        self.upstream[target].add(source)
        self.downstream[source].add(target)
    
    def get_upstream(self, table: str, recursive: bool = True) -> Set[str]:
        """Get all upstream dependencies."""
        if not recursive:
            return self.upstream.get(table, set())
        
        visited = set()
        to_visit = list(self.upstream.get(table, set()))
        
        while to_visit:
            current = to_visit.pop()
            if current not in visited:
                visited.add(current)
                to_visit.extend(self.upstream.get(current, set()))
        
        return visited
    
    def get_downstream(self, table: str, recursive: bool = True) -> Set[str]:
        """Get all downstream dependents."""
        if not recursive:
            return self.downstream.get(table, set())
        
        visited = set()
        to_visit = list(self.downstream.get(table, set()))
        
        while to_visit:
            current = to_visit.pop()
            if current not in visited:
                visited.add(current)
                to_visit.extend(self.downstream.get(current, set()))
        
        return visited
    
    def impact_analysis(self, table: str) -> dict:
        """Analyze impact of changes to a table."""
        downstream = self.get_downstream(table)
        return {
            "table": table,
            "direct_dependents": list(self.downstream.get(table, set())),
            "total_impacted": len(downstream),
            "all_impacted_tables": list(downstream)
        }

# Build example lineage graph
lineage = DataLineage()

# Source systems
lineage.add_table("raw_orders", {"layer": "bronze", "owner": "ingestion"})
lineage.add_table("raw_customers", {"layer": "bronze", "owner": "ingestion"})
lineage.add_table("raw_products", {"layer": "bronze", "owner": "ingestion"})

# Cleaned tables
lineage.add_lineage("raw_orders", "clean_orders")
lineage.add_lineage("raw_customers", "clean_customers")
lineage.add_lineage("raw_products", "clean_products")

# Aggregated tables
lineage.add_lineage("clean_orders", "order_facts")
lineage.add_lineage("clean_customers", "order_facts")
lineage.add_lineage("clean_products", "order_facts")

# Marts
lineage.add_lineage("order_facts", "sales_dashboard")
lineage.add_lineage("order_facts", "customer_360")
lineage.add_lineage("clean_customers", "customer_360")

# Impact analysis
print("Impact Analysis for 'raw_customers':")
print("-" * 40)
impact = lineage.impact_analysis("raw_customers")
print(f"Direct dependents: {impact['direct_dependents']}")
print(f"Total impacted tables: {impact['total_impacted']}")
print(f"All impacted: {impact['all_impacted_tables']}")

## Monitoring and Alerting Strategies

Effective data observability requires a well-designed alerting system:

### Alert Severity Levels

| Level | Description | Response Time | Examples |
|-------|-------------|---------------|----------|
| **Critical** | Business-impacting issues | Immediate | Production dashboard down, data loss |
| **High** | SLA violations | < 1 hour | Pipeline delays > 2 hours, data quality drops |
| **Medium** | Potential issues | Same day | Schema changes, volume anomalies |
| **Low** | Informational | Review weekly | Minor drift, performance degradation |

### Alert Best Practices

1. **Avoid alert fatigue**: Tune thresholds to minimize false positives
2. **Include context**: Provide links to dashboards, runbooks, and relevant data
3. **Route intelligently**: Send alerts to the right team based on table ownership
4. **Enable aggregation**: Group related alerts to reduce noise
5. **Track SLOs**: Define and monitor Service Level Objectives for data

In [ ]:
from enum import Enum
from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Callable, Optional
import json

class AlertSeverity(Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

class AlertStatus(Enum):
    ACTIVE = "active"
    ACKNOWLEDGED = "acknowledged"
    RESOLVED = "resolved"

@dataclass
class Alert:
    """Data quality alert."""
    id: str
    title: str
    description: str
    severity: AlertSeverity
    table: str
    check_type: str  # freshness, volume, schema, distribution
    created_at: datetime = field(default_factory=datetime.now)
    status: AlertStatus = AlertStatus.ACTIVE
    metadata: dict = field(default_factory=dict)
    
    def to_slack_message(self) -> dict:
        """Format alert for Slack notification."""
        emoji = {
            AlertSeverity.CRITICAL: "🔴",
            AlertSeverity.HIGH: "🟠",
            AlertSeverity.MEDIUM: "🟡",
            AlertSeverity.LOW: "🔵"
        }
        return {
            "text": f"{emoji[self.severity]} [{self.severity.value.upper()}] {self.title}",
            "blocks": [
                {
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": f"*{emoji[self.severity]} {self.title}*\n{self.description}"
                    }
                },
                {
                    "type": "context",
                    "elements": [
                        {"type": "mrkdwn", "text": f"*Table:* {self.table}"},
                        {"type": "mrkdwn", "text": f"*Check:* {self.check_type}"},
                        {"type": "mrkdwn", "text": f"*Time:* {self.created_at.isoformat()}"}
                    ]
                }
            ]
        }

class AlertManager:
    """Manage data quality alerts with routing and aggregation."""
    
    def __init__(self):
        self.alerts: List[Alert] = []
        self.handlers: List[Callable[[Alert], None]] = []
        self.owner_routing: Dict[str, str] = {}  # table -> owner
    
    def add_handler(self, handler: Callable[[Alert], None]):
        """Add alert handler (e.g., Slack, PagerDuty, email)."""
        self.handlers.append(handler)
    
    def set_table_owner(self, table: str, owner: str):
        """Set table ownership for routing."""
        self.owner_routing[table] = owner
    
    def fire_alert(self, alert: Alert):
        """Fire an alert and route to handlers."""
        self.alerts.append(alert)
        
        # Add owner to metadata
        if alert.table in self.owner_routing:
            alert.metadata["owner"] = self.owner_routing[alert.table]
        
        # Route to all handlers
        for handler in self.handlers:
            handler(alert)
    
    def get_active_alerts(self, severity: Optional[AlertSeverity] = None) -> List[Alert]:
        """Get active alerts, optionally filtered by severity."""
        active = [a for a in self.alerts if a.status == AlertStatus.ACTIVE]
        if severity:
            active = [a for a in active if a.severity == severity]
        return active

# Example usage
def log_handler(alert: Alert):
    print(f"ALERT: [{alert.severity.value}] {alert.title} - {alert.table}")

manager = AlertManager()
manager.add_handler(log_handler)
manager.set_table_owner("orders", "data-platform-team")

# Fire test alert
alert = Alert(
    id="alert-001",
    title="Orders table stale",
    description="Table has not been updated in 3 hours (SLA: 1 hour)",
    severity=AlertSeverity.HIGH,
    table="orders",
    check_type="freshness"
)

manager.fire_alert(alert)
print(f"\nActive alerts: {len(manager.get_active_alerts())}")

## Comprehensive Data Quality Monitor

A complete monitoring system that combines all pillars:

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Any, Dict, List
from datetime import datetime, timedelta
import uuid

@dataclass
class CheckResult:
    """Result of a data quality check."""
    check_name: str
    passed: bool
    message: str
    timestamp: datetime
    metadata: Dict[str, Any] = None

class DataQualityCheck(ABC):
    """Base class for data quality checks."""
    
    @abstractmethod
    def run(self) -> CheckResult:
        pass

class DataQualityOrchestrator:
    """Orchestrate multiple data quality checks."""
    
    def __init__(self, table_name: str, alert_manager: AlertManager = None):
        self.table_name = table_name
        self.checks: List[DataQualityCheck] = []
        self.results: List[CheckResult] = []
        self.alert_manager = alert_manager
    
    def add_check(self, check: DataQualityCheck):
        """Add a quality check to the orchestrator."""
        self.checks.append(check)
    
    def run_all(self) -> Dict[str, Any]:
        """Run all registered checks."""
        self.results = []
        
        for check in self.checks:
            result = check.run()
            self.results.append(result)
            
            # Fire alert if check failed
            if not result.passed and self.alert_manager:
                alert = Alert(
                    id=str(uuid.uuid4()),
                    title=f"{result.check_name} failed",
                    description=result.message,
                    severity=AlertSeverity.HIGH,
                    table=self.table_name,
                    check_type=result.check_name
                )
                self.alert_manager.fire_alert(alert)
        
        passed = sum(1 for r in self.results if r.passed)
        failed = len(self.results) - passed
        
        return {
            "table": self.table_name,
            "timestamp": datetime.now().isoformat(),
            "total_checks": len(self.results),
            "passed": passed,
            "failed": failed,
            "health_score": passed / len(self.results) if self.results else 1.0,
            "results": [
                {
                    "check": r.check_name,
                    "passed": r.passed,
                    "message": r.message
                } for r in self.results
            ]
        }

# Example: Create concrete checks
class MockFreshnessCheck(DataQualityCheck):
    def run(self) -> CheckResult:
        return CheckResult(
            check_name="freshness",
            passed=True,
            message="Table updated 15 minutes ago (SLA: 1 hour)",
            timestamp=datetime.now()
        )

class MockVolumeCheck(DataQualityCheck):
    def run(self) -> CheckResult:
        return CheckResult(
            check_name="volume",
            passed=False,
            message="Row count 5,000 is 50% below expected (10,000)",
            timestamp=datetime.now()
        )

class MockSchemaCheck(DataQualityCheck):
    def run(self) -> CheckResult:
        return CheckResult(
            check_name="schema",
            passed=True,
            message="No schema changes detected",
            timestamp=datetime.now()
        )

# Run orchestrator
orchestrator = DataQualityOrchestrator("orders", alert_manager=manager)
orchestrator.add_check(MockFreshnessCheck())
orchestrator.add_check(MockVolumeCheck())
orchestrator.add_check(MockSchemaCheck())

report = orchestrator.run_all()

print("\nData Quality Report:")
print("=" * 50)
print(f"Table: {report['table']}")
print(f"Health Score: {report['health_score']:.0%}")
print(f"Checks: {report['passed']}/{report['total_checks']} passed")
print("\nDetails:")
for r in report['results']:
    status = "✅" if r['passed'] else "❌"
    print(f"  {status} {r['check']}: {r['message']}")

## Data Observability Tools Overview

### Enterprise Tools

| Tool | Type | Key Features | Best For |
|------|------|--------------|----------|
| **Monte Carlo** | Platform | ML-powered anomaly detection, end-to-end lineage, incident management | Enterprise data teams |
| **Datadog Data Jobs** | Observability | Pipeline monitoring, cost tracking, integrates with APM | DevOps-oriented teams |
| **Atlan** | Catalog + Observability | Active metadata, lineage, data discovery | Data governance focus |
| **Soda** | Testing + Monitoring | SQL-based checks, SodaCL language, Soda Cloud | Developer-first teams |
| **Great Expectations** | Open Source | Python-native, extensive expectations library | Data engineering teams |
| **dbt Tests** | Built-in | Schema tests, freshness, relationships | dbt users |

### Comparison Matrix

```
                    Monte Carlo  Datadog   Atlan   Soda    Great Expectations  dbt
Freshness               ✅          ✅       ✅      ✅            ✅           ✅
Volume                  ✅          ✅       ✅      ✅            ✅           ⚠️
Schema                  ✅          ⚠️       ✅      ✅            ✅           ✅
Distribution            ✅          ⚠️       ⚠️      ✅            ✅           ⚠️
Lineage                 ✅          ⚠️       ✅      ⚠️            ❌           ✅
ML Anomaly Detection    ✅          ✅       ⚠️      ✅            ⚠️           ❌
Self-hosted             ❌          ❌       ❌      ✅            ✅           ✅
```

✅ = Full support | ⚠️ = Partial/Limited | ❌ = Not available

### Great Expectations Example

Great Expectations is a popular open-source tool for data validation:

In [ ]:
# Great Expectations-style validation (conceptual example)
# In practice, you would use the actual great_expectations library

class ExpectationSuite:
    """Simplified Great Expectations-style validator."""
    
    def __init__(self, name: str):
        self.name = name
        self.expectations = []
        self.results = []
    
    def expect_column_values_to_not_be_null(self, column: str, mostly: float = 1.0):
        self.expectations.append({
            "type": "not_null",
            "column": column,
            "mostly": mostly
        })
        return self
    
    def expect_column_values_to_be_between(self, column: str, min_val: float, max_val: float):
        self.expectations.append({
            "type": "between",
            "column": column,
            "min": min_val,
            "max": max_val
        })
        return self
    
    def expect_column_values_to_be_unique(self, column: str):
        self.expectations.append({
            "type": "unique",
            "column": column
        })
        return self
    
    def expect_table_row_count_to_be_between(self, min_count: int, max_count: int):
        self.expectations.append({
            "type": "row_count_between",
            "min": min_count,
            "max": max_count
        })
        return self
    
    def validate(self, df: pd.DataFrame) -> dict:
        """Validate dataframe against expectations."""
        results = []
        
        for exp in self.expectations:
            if exp["type"] == "not_null":
                null_rate = df[exp["column"]].isnull().mean()
                passed = (1 - null_rate) >= exp["mostly"]
                results.append({
                    "expectation": f"{exp['column']} not null",
                    "passed": passed,
                    "observed": f"{(1-null_rate):.1%} non-null"
                })
            
            elif exp["type"] == "between":
                col = df[exp["column"]]
                in_range = ((col >= exp["min"]) & (col <= exp["max"])).all()
                results.append({
                    "expectation": f"{exp['column']} between {exp['min']} and {exp['max']}",
                    "passed": in_range,
                    "observed": f"min={col.min():.2f}, max={col.max():.2f}"
                })
            
            elif exp["type"] == "unique":
                is_unique = df[exp["column"]].is_unique
                results.append({
                    "expectation": f"{exp['column']} is unique",
                    "passed": is_unique,
                    "observed": f"{df[exp['column']].nunique()} unique of {len(df)} rows"
                })
            
            elif exp["type"] == "row_count_between":
                count = len(df)
                passed = exp["min"] <= count <= exp["max"]
                results.append({
                    "expectation": f"row count between {exp['min']} and {exp['max']}",
                    "passed": passed,
                    "observed": f"{count} rows"
                })
        
        passed_count = sum(1 for r in results if r["passed"])
        return {
            "suite": self.name,
            "success": passed_count == len(results),
            "passed": passed_count,
            "failed": len(results) - passed_count,
            "results": results
        }

# Example usage
df = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "name": ["Alice", "Bob", None, "David", "Eve"],
    "amount": [100.5, 250.0, 75.25, 500.0, 125.75],
    "age": [25, 30, 35, 200, 28]  # 200 is invalid
})

suite = ExpectationSuite("orders_validation")
suite.expect_column_values_to_not_be_null("id")
suite.expect_column_values_to_not_be_null("name", mostly=0.8)
suite.expect_column_values_to_be_unique("id")
suite.expect_column_values_to_be_between("amount", 0, 1000)
suite.expect_column_values_to_be_between("age", 0, 120)
suite.expect_table_row_count_to_be_between(1, 100)

validation_result = suite.validate(df)

print(f"Suite: {validation_result['suite']}")
print(f"Overall Success: {validation_result['success']}")
print(f"Results: {validation_result['passed']}/{validation_result['passed'] + validation_result['failed']} passed")
print("\nDetails:")
for r in validation_result['results']:
    status = "✅" if r['passed'] else "❌"
    print(f"  {status} {r['expectation']}: {r['observed']}")

### Soda-style SQL Checks

Soda uses a YAML-based configuration with SQL under the hood:

In [ ]:
# Soda-style check configuration (conceptual)
soda_checks_yaml = """
# SodaCL Check Definition
checks for orders:
  # Freshness check
  - freshness(updated_at) < 1h
  
  # Volume checks
  - row_count > 0
  - row_count between 9000 and 11000
  
  # Validity checks
  - missing_count(order_id) = 0
  - duplicate_count(order_id) = 0
  
  # Distribution checks
  - avg(order_amount) between 80 and 120
  - max(order_amount) < 10000
  
  # Schema check
  - schema:
      name: Check schema
      fail:
        when missing column:
          - order_id
          - customer_id
          - order_amount
        when wrong type:
          order_amount: decimal
"""

print("Soda Check Configuration Example:")
print(soda_checks_yaml)

## Implementing Data Observability: Best Practices

### 1. Start with Critical Pipelines
- Identify business-critical data assets
- Implement freshness and volume checks first
- Gradually expand coverage

### 2. Define SLAs and SLOs
- **SLA (Service Level Agreement)**: External commitment to stakeholders
- **SLO (Service Level Objective)**: Internal target (typically stricter)
- **SLI (Service Level Indicator)**: Actual metric being measured

### 3. Build Data Contracts
- Define expected schema, freshness, and quality rules
- Version contracts alongside code
- Enforce contracts in CI/CD pipelines

### 4. Implement Circuit Breakers
- Stop pipeline execution when quality degrades
- Prevent bad data from propagating downstream

### 5. Create Runbooks
- Document common failure scenarios
- Include resolution steps
- Link runbooks from alerts

In [ ]:
class CircuitBreaker:
    """Pipeline circuit breaker for data quality failures."""
    
    def __init__(self, failure_threshold: int = 3, reset_timeout_minutes: int = 30):
        self.failure_threshold = failure_threshold
        self.reset_timeout = timedelta(minutes=reset_timeout_minutes)
        self.failures = 0
        self.last_failure_time: Optional[datetime] = None
        self.state = "closed"  # closed, open, half-open
    
    def record_success(self):
        """Record successful check."""
        self.failures = 0
        self.state = "closed"
    
    def record_failure(self):
        """Record failed check."""
        self.failures += 1
        self.last_failure_time = datetime.now()
        
        if self.failures >= self.failure_threshold:
            self.state = "open"
    
    def can_proceed(self) -> bool:
        """Check if pipeline can proceed."""
        if self.state == "closed":
            return True
        
        if self.state == "open":
            # Check if reset timeout has passed
            if datetime.now() - self.last_failure_time > self.reset_timeout:
                self.state = "half-open"
                return True
            return False
        
        # half-open: allow one attempt
        return True
    
    def status(self) -> dict:
        return {
            "state": self.state,
            "failures": self.failures,
            "can_proceed": self.can_proceed()
        }

# Example: Circuit breaker in action
breaker = CircuitBreaker(failure_threshold=3)

# Simulate failures
print("Simulating quality check failures:")
for i in range(5):
    if breaker.can_proceed():
        print(f"  Check {i+1}: Running... (state: {breaker.state})")
        breaker.record_failure()  # Simulate failure
    else:
        print(f"  Check {i+1}: BLOCKED - Circuit breaker open")

print(f"\nFinal status: {breaker.status()}")

## Key Takeaways

### Core Concepts

1. **Five Pillars**: Freshness, Volume, Schema, Distribution, and Lineage provide comprehensive data health visibility

2. **Proactive Monitoring**: Detect issues before they impact downstream consumers and business decisions

3. **Automated Detection**: Use statistical methods (z-scores, KS tests) for anomaly detection at scale

4. **Impact Analysis**: Lineage enables understanding of upstream dependencies and downstream impact

5. **Alert Management**: Proper severity levels, routing, and aggregation prevent alert fatigue

### Implementation Strategy

| Phase | Focus | Activities |
|-------|-------|------------|
| **1. Foundation** | Critical assets | Deploy freshness/volume checks on key tables |
| **2. Expansion** | Broader coverage | Add schema/distribution monitoring, build lineage |
| **3. Automation** | Self-service | ML-based anomaly detection, automated remediation |
| **4. Governance** | Data contracts | SLAs, ownership, quality certification |

### Tool Selection Criteria

- **Scale**: How many tables/pipelines need monitoring?
- **Complexity**: Custom checks vs. out-of-box detection?
- **Integration**: Does it work with your data stack?
- **Cost**: Open-source vs. managed service tradeoffs?
- **Team Skills**: SQL-based vs. Python-based configuration?

### Remember

> "You can't fix what you can't see. Data observability is not optional—it's essential for building trust in your data platform."

---

**Related Topics**: Data Quality, Data Contracts, DataOps, MLOps